In [1]:
!pip install flask flask-restful dash pandas plotly
!pip install lightfm

from flask import Flask, request, jsonify
import joblib
import numpy as np


INFO: pip is looking at multiple versions of dash to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of dash to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808328 sha256=8d3948cbdb09112ca5e1eaa0e55cf63b631a54bc0eeb63000057f6b01d1f4437
  Stored in directory: /ro

### API for Recommendations


In [2]:
from flask import Flask, request, jsonify
import joblib
import numpy as np
import pandas as pd

# Define file paths for models and data
file_paths = {
    'model': 'lightfm_model.joblib',
    'user_map': 'user_map.joblib',
    'product_map': 'product_map.joblib',
    'reverse_user_map': 'reverse_user_map.joblib',
    'reverse_product_map': 'reverse_product_map.joblib',
    'interaction_matrix': 'interaction_matrix.joblib',
    'cosine_similarity': 'tfidf_cosine_similarity.joblib',
    'product_metadata': 'product_metadata.csv',
    'segmented_customers': 'segmented_customers.csv',
}

# Load model and data
model = joblib.load(file_paths['model'])
user_map = joblib.load(file_paths['user_map'])
product_map = joblib.load(file_paths['product_map'])
reverse_product_map = joblib.load(file_paths['reverse_product_map'])
reverse_user_map = joblib.load(file_paths['reverse_user_map'])
interaction_matrix = joblib.load(file_paths['interaction_matrix'])
cosine_sim = joblib.load(file_paths['cosine_similarity'])
product_metadata = pd.read_csv(file_paths['product_metadata'])
segmented_customers = pd.read_csv(file_paths['segmented_customers'])

# Flask setup
app = Flask(__name__)

# Recommendation function (reuse hybrid logic)
def get_recommendations(user_id, num_recommendations=5):
    if user_id not in user_map:
        return {"error": "User not found."}
    user_idx = user_map[user_id]
    scores = model.predict(user_idx, np.arange(interaction_matrix.shape[1]))
    top_items = np.argsort(-scores)[:num_recommendations]
    recommendations = [reverse_product_map[idx] for idx in top_items]
    return recommendations

@app.route('/recommend', methods=['GET'])
def recommend():
    user_id = int(request.args.get('user_id'))
    recommendations = get_recommendations(user_id)
    return jsonify({"user_id": user_id, "recommendations": recommendations})

if __name__ == '__main__':
    print("Starting Flask app...")
    app.run(debug=True, host='0.0.0.0', port=8000)



Starting Flask app...
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
